In [65]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
from dataset import AirplaneDataset
from matplotlib.pyplot import imshow
import numpy as np
%matplotlib inline

In [66]:
# !unzip "/content/Retail Pulse ML Assignment Data.zip"

In [67]:
# set datapath to the 'Retail Pulse ML Assignment' folder
DATAPATH = '/content/'

train_dataset, validate_dataset = AirplaneDataset(phase='train'), AirplaneDataset(phase='val')

train_loader, validate_loader = DataLoader(train_dataset, batch_size=100, shuffle=True), DataLoader(validate_dataset, batch_size=100)

num_classes = train_dataset.num_classes

In [97]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Linear layers
        self.dropout1 = nn.Dropout()
        self.fc1 = nn.Linear(8 * 18 * 18, 1024)
        self.relu4 = nn.ReLU()
        self.dropout2 = nn.Dropout()
        self.fc2 = nn.Linear(1024, 256)
        self.relu5 = nn.ReLU()
        self.dropout3 = nn.Dropout()
        self.fc3 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        # Max pool 2
        out = self.maxpool2(out)
        
        out = out.view(out.size(0), -1)
        
        # Linear
        # out = self.dropout1(out)
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.dropout2(out)
        out = self.fc2(out)
        out = self.relu5(out)
        out = self.dropout3(out)
        out = self.fc3(out)
        
        return out

In [98]:
# Prepare the model, loss, optimizer
model = CNNModel()
if torch.cuda.is_available():
  model.cuda()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [99]:
# Verify the parameter shapes
print(model.parameters())
print(len(list(model.parameters())))

for i in range(len(list(model.parameters()))):
  print(list(model.parameters())[i].size())

<generator object Module.parameters at 0x7fd7faed0ba0>
10
torch.Size([32, 3, 3, 3])
torch.Size([32])
torch.Size([8, 32, 3, 3])
torch.Size([8])
torch.Size([1024, 2592])
torch.Size([1024])
torch.Size([256, 1024])
torch.Size([256])
torch.Size([70, 256])
torch.Size([70])


In [100]:
# Set the iterations, epochs
n_iters = 3000
batch_size = 100
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [101]:
iter = 0
    
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):

        if torch.cuda.is_available():
          images = Variable(images.cuda())
          labels = Variable(labels.cuda())
        else:
          images = Variable(images)
          labels = Variable(labels)
        
        optimizer.zero_grad()
        
        model.train()
        outputs = model(images)

        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()

        iter += 1
        
        if iter%100 == 0:
            correct = 0
            total = 0
            for images, labels in validate_loader:
                if torch.cuda.is_available():
                  images = Variable(images.cuda())
                else:
                  images = Variable(images)
                
                model.eval()
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                  correct += (predicted==labels).sum()

            accuracy = 100 * correct / total
            
            print("Iteration: {}. Loss {}. Accuracy: {}".format(iter, loss.item(), accuracy))
            

Iteration: 100. Loss 3.9200754165649414. Accuracy: 7
Iteration: 200. Loss 4.027720928192139. Accuracy: 8
Iteration: 300. Loss 3.6873257160186768. Accuracy: 9
Iteration: 400. Loss 3.227139949798584. Accuracy: 14
Iteration: 500. Loss 3.386666774749756. Accuracy: 15
Iteration: 600. Loss 2.916623592376709. Accuracy: 19
Iteration: 700. Loss 2.8206565380096436. Accuracy: 22
Iteration: 800. Loss 2.5876004695892334. Accuracy: 24
Iteration: 900. Loss 2.144606590270996. Accuracy: 29
Iteration: 1000. Loss 2.385101079940796. Accuracy: 29
Iteration: 1100. Loss 1.8491215705871582. Accuracy: 31
Iteration: 1200. Loss 1.5711586475372314. Accuracy: 34
Iteration: 1300. Loss 1.4156100749969482. Accuracy: 35
Iteration: 1400. Loss 1.1908375024795532. Accuracy: 35
Iteration: 1500. Loss 1.0558358430862427. Accuracy: 35
Iteration: 1600. Loss 0.8028101921081543. Accuracy: 37
Iteration: 1700. Loss 0.7273221015930176. Accuracy: 34
Iteration: 1800. Loss 0.9163447618484497. Accuracy: 38
Iteration: 1900. Loss 0.6586

In [102]:
# Post training calculate total accuracy, classwise accuracies
classwise_accuracy = {}

for i in range(70):
  classwise_accuracy[i] = (0,0)

for images, labels in validate_loader:

    if torch.cuda.is_available():
      images = Variable(images.cuda())
    else:
      images = Variable(images)
    
    model.eval()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)

    labels_list = labels.tolist()
    pred_list = predicted.tolist()

    for i in range(predicted.size(0)):
      (x,y) = classwise_accuracy[labels_list[i]]
      if (pred_list[i]==labels_list[i]):
        x = x + 1
      y = y+1
      classwise_accuracy[labels_list[i]] = (x,y)
    
    if torch.cuda.is_available():
        correct += (predicted.cpu() == labels.cpu()).sum()
    else:
      correct += (predicted==labels).sum()

accuracy = 100 * correct / total

print("Final: Accuracy: {}\n".format(accuracy))

# print classwise accuracies with class name
with open(os.path.join(DATAPATH,'families.txt'),'r') as f:
    for idx,line in enumerate(f.readlines()):
        (x,y) = classwise_accuracy[idx]
        print("Class :{}, Class name :{}, Accuracy :{}".format(idx,line.strip(),100 * x/y))


Final: Accuracy: 40

Class :0, Class name :A300, Accuracy :12.121212121212121
Class :1, Class name :A310, Accuracy :26.470588235294116
Class :2, Class name :A320, Accuracy :44.3609022556391
Class :3, Class name :A330, Accuracy :32.83582089552239
Class :4, Class name :A340, Accuracy :39.849624060150376
Class :5, Class name :A380, Accuracy :51.515151515151516
Class :6, Class name :ATR-42, Accuracy :41.1764705882353
Class :7, Class name :ATR-72, Accuracy :30.303030303030305
Class :8, Class name :An-12, Accuracy :48.484848484848484
Class :9, Class name :BAE 146, Accuracy :41.791044776119406
Class :10, Class name :BAE-125, Accuracy :33.333333333333336
Class :11, Class name :Beechcraft 1900, Accuracy :32.35294117647059
Class :12, Class name :Boeing 707, Accuracy :18.181818181818183
Class :13, Class name :Boeing 717, Accuracy :15.151515151515152
Class :14, Class name :Boeing 727, Accuracy :20.58823529411765
Class :15, Class name :Boeing 737, Accuracy :56.390977443609025
Class :16, Class name 

In [103]:
# Save the model
torch.save(model.state_dict(), 'best_model.pt')